In [1]:
import numpy as np
import pandas

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import csv

In [2]:
from uci_datasets import Dataset

In [3]:
data_name = "buzz"
data = Dataset(data_name)

buzz dataset, N=583250, d=77


In [4]:
X = data.x
Y = data.y
#X = data[:,:-1]
#Y = data[:, -1].reshape(-1,1)

print(X)
print(Y)

ind = np.arange(X.shape[0])
print(ind)
prop = 0.9
if data_name == "buzz":
    prop = 6/9
n = int(X.shape[0] * prop)
print(n)

[[  42.66   90.23  122.32 ...  130.4   118.29  110.06]
 [1178.7  1134.2   597.32 ... 1137.4  1151.3  1050.1 ]
 [-135.34 -133.77 -159.68 ... -199.6  -209.71 -197.94]
 ...
 [ 533.66  709.23  714.32 ...  809.4   737.29  860.06]
 [-139.34 -136.77 -159.68 ... -199.6  -220.71 -218.94]
 [-138.34 -132.77 -158.68 ... -195.6  -220.71 -216.94]]
[[ 2.163  ]
 [ 3.6614 ]
 [-0.45896]
 ...
 [ 3.6842 ]
 [-2.7103 ]
 [-1.052  ]]
[     0      1      2 ... 583247 583248 583249]
388833


In [5]:
from gpmaniflow.models.BezierProcess2 import BezierProcess
from gpflow.likelihoods import Gaussian
from gpflow.models import SGPR
from gpflow.inducing_variables import InducingPoints
import gpflow

import tensorflow as tf
import tensorflow_probability as tfp

from gpflow.utilities import print_summary

2022-05-15 11:53:29.244901: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-15 11:53:29.244960: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
BezierRMSE = np.zeros(3)
BezierRMedSE = np.zeros(3)
BezierTestLL = np.zeros(3)


ITERATIONS = 10000
ORDER = 20
BATCH_SIZE = 500
INPUT_DIM = X.shape[1]

for i in range(3):
    print('Split', i + 1, 'of', 20)
    np.random.seed(666 + i) # The devil's seed
    np.random.shuffle(ind)
    
    ### GET TRAINING SPLIT AND PREPROCESS ###
    
    Xtrain = X[ind[:n], :]; Ytrain = Y[ind[:n], :]
    Xtest = X[ind[n:], :]; Ytest = Y[ind[n:], :]
    
    mi, ma = Xtrain.min(axis=0), Xtrain.max(axis=0)
    mami = ma - mi
    mami = np.where(mami > 1e-6, mami, 1.) # Don't divide by zero
    Xtrain = (Xtrain - mi) / mami
    Xtest = (Xtest - mi) / mami

    mi = 0.01
    ma = 0.99
    mami2 = np.where(mami > 1e-6, mami, 0.)
    Xtrain = Xtrain * (ma - mi) + mi
    Xtest = Xtest * (ma - mi) + mi
    print(Xtest.max())
    print(Xtest.min())
    print(Xtest.shape)
    Xtest21 = Xtest[Xtest.min(axis=1)>-0.01,:]
    Xtest2 = Xtest21[Xtest21.max(axis=1)<1.01,:]
    print(Xtest2.shape)
    print(Xtest2.max())
    print(Xtest2.min())
    meany, stdy = np.average(Ytrain), np.std(Ytrain)
    Ytrain = (Ytrain - meany) / stdy
    Ytest2 = Ytest[Xtest.min(axis=1)>-0.01,:]
    Ytest2 = Ytest2[Xtest21.max(axis=1)<1.01,:]
    print(Ytest2.shape)
    ############## BEZIER GP ##################
    
    m = BezierProcess(input_dim = INPUT_DIM, orders = ORDER, likelihood = Gaussian(), num_data = Xtrain.shape[0])
    m.likelihood.variance.assign(1.0)
    gpflow.set_trainable(m.likelihood.variance, False)
    
    minibatch_size = BATCH_SIZE

    train_dataset = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain)).repeat().shuffle(Xtrain.shape[0])
    train_iter = iter(train_dataset.batch(minibatch_size))
    
    training_loss = m.training_loss_closure(train_iter, compile=True)
    optimizer = tf.optimizers.Adam(learning_rate = 0.001)

    @tf.function
    def optimization_step():
        optimizer.minimize(training_loss, m.trainable_variables)

    iterations = ITERATIONS
    print('Training....')
    for step in range(iterations):
        optimization_step()
        if step % 500 == 0:
            elbo = -training_loss().numpy()
            pr, _ = m.predict_f(Xtest[:100,:])
            batchrmse = np.sqrt(np.mean((Ytest[:100,:] - (meany + stdy*pr.numpy()))**2))
            print('Split:', i + 1, 'Step:', step, 'Elbo:', elbo)
            print('RMSE:', batchrmse)
    
    gpflow.set_trainable(m.BN, False) # FIX VARIATIONAL PARAMS
    gpflow.set_trainable(m.likelihood.variance, True) # AND TRAIN ONLY LIKELIHOOD VARIANCE
    
    training_loss = m.training_loss_closure(train_iter, compile=True)
    optimizer = tf.optimizers.Adam(learning_rate = 0.01)

    @tf.function
    def optimization_step():
        optimizer.minimize(training_loss, m.trainable_variables)
    for step in range(iterations):
        optimization_step()
        if step % 500 == 0:
            elbo = -training_loss().numpy()
            pr, _ = m.predict_f(Xtest[:100,:])
            batchrmse = np.sqrt(np.mean((Ytest[:100,:] - (meany + stdy*pr.numpy()))**2))
            print('Split:', i + 1, 'Step:', step, 'Elbo:', elbo)
            print('RMSE:', batchrmse)
    
    
    test_dataset = tf.data.Dataset.from_tensor_slices((Xtest2, Ytest2))
    test_iter = iter(test_dataset.batch(minibatch_size))
    print('Testing...')
    SE = 0.
    loglik = 0.
    for batch in test_iter:
        xb = batch[0]
        yb = batch[1]
        preds, predvar = m.predict_f(xb)
        SE += np.sum((yb - (meany + stdy*preds.numpy()))**2)
        
        pred_mean, pred_var = m.predict_y(xb)
        pred_dist = tfp.distributions.Normal(loc=meany + pred_mean*stdy, scale=stdy*pred_var**0.5)
        loglik += tf.reduce_sum(pred_dist.log_prob(yb)).numpy()
        
    RMSE = np.sqrt( SE / Xtest.shape[0] )
    loglik = loglik / Xtest.shape[0]
    print('RMSE:', RMSE)
    print('Test log-likelihood:', loglik)
    ### Append results
    BezierRMSE[i] = RMSE
    BezierRMedSE[i] = 0
    BezierTestLL[i] = loglik
    
print('Experiment concluded!')

Split 1 of 20
1.5930762603557576
0.01
(194417, 77)
(194409, 77)
0.9955368514185922
0.01
(194409, 1)


2022-05-15 11:53:47.629913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-15 11:53:47.631611: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-15 11:53:47.632307: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (martinj): /proc/driver/nvidia/version does not exist
2022-05-15 11:53:47.643076: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-15 11:53:47.967024: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but 

Training....


2022-05-15 11:54:04.172164: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-15 11:54:24.375496: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 220095 of 388833
2022-05-15 11:54:34.321140: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 239110 of 388833
2022-05-15 11:54:44.310664: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 242779 of 388833
2022-05-15 11:54:54.340200: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 242838 of 388833
2022-05-15 11:55:04.309911: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 260533 of 388833
2022-05-15 11:55:14.385152: I tensorflow/core/kernels/data/shuffle_dataset

KeyboardInterrupt: 

In [ ]:
print('Iterations:', ITERATIONS)
print('Order:', ORDER)
print('Batch size:', BATCH_SIZE)
print('Average RMSE:', np.mean(BezierRMSE), 'Standard deviation:', np.std(BezierRMSE))
print('Average RMedSE:', np.mean(BezierRMedSE), 'Standard deviation:', np.std(BezierRMedSE))
print('Average test-ll:', np.mean(BezierTestLL), 'Standard deviation:', np.std(BezierTestLL))